In [ ]:
# t3_numeric.py  — baseline T=3 numeric (bisection) and comparison to T=2
import os, numpy as np, matplotlib.pyplot as plt
# Ensure output directory exists for figures
os.makedirs("figures", exist_ok=True)

# ---- Baseline params (same as your figures) ----
# p: prior that LEFT arm is good; s: success probability when the good arm is pulled; V: prize value.
# c, r: thinking cost and breakthrough probability used for baseline numeric point and sweep anchors.
p, s, V = 0.6, 0.7, 1.0
c, r    = 0.10, 0.40

def Lambda(p, s, V, r):
    """One-period continuation value: Λ = s*V*[ r + (1 - r)*p ]."""
    # continuation value with one period left
    return s * V * (r + (1 - r) * p)

# ---- T=2 payoffs (for reference) ----
def U_D_T2(p, s, V):                # total across both periods
    """T=2: payoff from Do at t=1, including period-2 consequences when both Do."""
    return p * s * V
def U_T_T2(x, p, s, V, c, r):       # depends on opponent's Do prob x
    """T=2: payoff from Think at t=1 against opponent Do prob x."""
    return -c + 0.5 * (1 + x) * Lambda(p, s, V, r)

# ---- T=3 payoffs under the baseline extension ----
# Decisions only at t=1; exclusivity applies only to period 2; period 3 is contestable.
# => period-3 adds the same 0.5*p*s*V to BOTH actions, so the mix condition is unchanged.
def U_D_T3(p, s, V):
    """T=3 baseline extension: Do payoff equals T=2 Do plus the symmetric period-3 add-on."""
    return U_D_T2(p, s, V) + 0.5 * p * s * V
def U_T_T3(x, p, s, V, c, r):
    """T=3 baseline extension: Think payoff equals T=2 Think plus the same period-3 add-on."""
    return U_T_T2(x, p, s, V, c, r) + 0.5 * p * s * V

# ---- Bisection to solve U_D = U_T(x) on [0,1] with corners ----
def mix_bisection_T3(p, s, V, c, r, tol=1e-9):
    """
    Compute the symmetric t=1 equilibrium for the T=3 baseline via bisection on x in [0,1].
    Corners mirror T=2 because period-3 adds a constant to both actions.
    Returns (x_do, 1 - x_do, region).
    """
    UD = U_D_T3(p, s, V)
    # Corner checks (same as T=2 in this baseline)
    L = Lambda(p, s, V, r)
    if (p*s*V + c) <= 0.5 * L:   # always-Think
        return 0.0, 1.0, "always-think"
    if (p*s*V + c) >= L:         # always-Do
        return 1.0, 0.0, "always-do"
    # Interior: root of f(x) = U_T_T3(x) - UD
    lo, hi = 0.0, 1.0
    for _ in range(80):
        mid = 0.5 * (lo + hi)
        fmid = U_T_T3(mid, p, s, V, c, r) - UD
        if fmid == 0 or (hi - lo) < tol:
            x = mid
            break
        # Evaluate at boundaries to decide which half to keep
        flo = U_T_T3(lo, p, s, V, c, r) - UD
        if np.sign(flo) == np.sign(fmid):
            lo = mid
        else:
            hi = mid
    x = 0.5 * (lo + hi)
    return float(x), float(1.0 - x), "interior"

# Solve once at the baseline parameters for a sanity check
x_do_T3, tf_T3, reg_T3 = mix_bisection_T3(p, s, V, c, r)
print(f"T=3 baseline: x*={x_do_T3:.3f}, Think freq={tf_T3:.3f}, region={reg_T3}")

# ---- Compare to T=2 across r (you'll see they coincide) ----
def xstar_T2(p, s, V, c, r):
    """Closed-form T=2 equilibrium (x_do, 1 - x_do) as a function of r for reference lines."""
    L = Lambda(p, s, V, r)
    # corners
    if (p*s*V + c) <= 0.5 * L:      return 0.0, 1.0
    if (p*s*V + c) >= L:            return 1.0, 0.0
    x = (2.0 * (p*s*V + c) / L) - 1.0
    x = float(np.clip(x, 0.0, 1.0))
    return x, 1.0 - x

def xstar_T3_numeric(p, s, V, c, r):
    """Wrapper to return only (x_do, 1 - x_do) from the T=3 bisection solver."""
    return mix_bisection_T3(p, s, V, c, r)[0:2]  # (x_do, think_freq)

# r-sweep: show that T=3 baseline Think coincides with T=2 across r
r_grid = np.linspace(0.05, 0.99, 120)
tf2 = [xstar_T2(p, s, V, c, r)[1] for r in r_grid]
tf3 = [xstar_T3_numeric(p, s, V, c, r)[1] for r in r_grid]

# Plot and save figure comparing T=2 closed-form to T=3 numeric
plt.figure(figsize=(7,4))
plt.plot(r_grid, tf2, label="T=2 (theory)")
plt.plot(r_grid, tf3, linestyle="--", label="T=3 (numeric)")
plt.axvline(x = (c / (s*V*(1-p))), linestyle=":", color="k", lw=1)  # r† threshold for T=2
plt.xlabel("Breakthrough probability (r)")
plt.ylabel("Think frequency (1 - x*)")
plt.title("Period-1 Think: T=2 vs T=3 (baseline extension)")
plt.legend()
plt.tight_layout()
plt.savefig("figures/t3_vs_t2_think_vs_r.png", dpi=200)
plt.close()
print("Saved figures/t3_vs_t2_think_vs_r.png")


T=3 baseline: x*=0.955, Think freq=0.045, region=interior
Saved figures/t3_vs_t2_think_vs_r.png
